In [1]:
import asyncio
import json
import logging
import websockets

logging.basicConfig()

usersWithoutID = set() # Array für User ohne zugeordnete ID

usersWithID = set() # Array für User mit ID

# Klasse für eine Nachricht
class message:
    session_id = 0
    type = ''
    message = ''
    send_time = ''
    def __init__(self, id, type, message, time):
        self.session_id = id
        self.type = type
        self.message = message
        self.send_time = time

# Klasse für einen User
class user:
    session_id = 0
    canControl = False
    def __init__(self, id, control, websocket):
        self.session_id = id
        self.canControl = control
        self.websocket = websocket


In [ ]:
def state_event():
    return json.dumps({"type": "state", **STATE})

def users_event():
    return json.dumps({"type": "users", "count": len(USERS)})

async def notify_state():
    if USERS:
        message = state_event()
        await asyncio.wait([user.send(message) for user in USERS])

async def notify_users():
    if USERS:
        message = users_event()
        await asyncio.wait([user.send(message) for user in USERS])

# Registriere neuen Nutzer ohne ID
async def registerUninitUser(websocket):
    uninitUsers.add(websocket)

# Registriere neuen Nutzer mit ID
async def registerInitUser(websocket, id):
    newUser = user(id, websocket)
    initedUsers.add(newUser)
    unregisterUninitUser(websocket)

# Entferne Nutzer ohne ID
async def unregisterUninitUser(websocket):
    uninitUsers.remove(websocket)

# Entferne Nutzer mit ID
async def unregisterInitUser(user):
    initedUsers.remove(user)

# Auswerten der empfangenen Nachricht
async def evaluateMessage(message, websocket):
    # Registrieren als neuer Nutzer mit ID
    if data["type"] == "register":
        registerInitUser(websocket, data["id"])
    elif data["type"] == "ping":
        
        await notify_state()
    else:
        logging.error("unsupported event: {}", data)
    
async def wsServer(websocket, path):
    await registerUninitUser(websocket)
    try:
        await websocket.send(state_event())
        async for message in websocket:
            data = json.loads(message)
            evaluateMessage(data, websocket)
    finally:
        await unregister(websocket)

In [2]:
start_server = websockets.serve(wsServer, "localhost", 6789)

asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()

RuntimeError: This event loop is already running